In [1]:
import sys
import os
import time
import numpy as np
import math

pyngp_path = '/cluster/home/jpostels/nnaimi/instant-ngp/build'
sys.path.append(pyngp_path)
sys.path.append(os.path.abspath("/cluster/home/jpostels/nnaimi/instant-ngp/scripts"))
import pyngp as ngp
from scenes import *

In [2]:
!nvidia-smi

Mon Aug 21 10:28:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:A1:00.0 Off |                  N/A |
|  0%   28C    P8    41W / 370W |      1MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
def get_scene(scene):
    for scenes in [scenes_sdf, scenes_nerf, scenes_image, scenes_volume]:
        if scene in scenes:
            return scenes[scene]
    return None

In [4]:
# testbed = ngp.Testbed()
testbed = ngp.Testbed()

10:28:12 SUCCESS  Initialized CUDA. Active GPU is #0: NVIDIA GeForce RTX 3090 [86]


In [ ]:
snapshot = '/cluster/work/cvl/jpostels/nnaimi/data/kodim11-20230721-232433.ingp'
# snapshot = '/cluster/work/cvl/jpostels/nnaimi/data/snapshot_8bit_params.ingp'

In [ ]:
scene_info = get_scene(snapshot)
if scene_info is not None:
    args.load_snapshot = default_snapshot_filename(scene_info)
testbed.load_snapshot(snapshot)

In [ ]:
# scene_info_og = get_scene(testbed_og)
# testbed_og.load_snapshot(snapshot_og)

# scene_info = get_scene(snapshot)

# testbed.load_snapshot(snapshot)

# level_params = [testbed.n_encoding_level_params(l) for l in range(4)]
# print(testbed.n_params())
# print(level_params)

In [ ]:
??testbed.render

In [ ]:
# image rendering
out_dir = "/cluster/work/cvl/jpostels/nnaimi/data/"
out_name = "image_8bits"

print(f"Rendering {out_name}.png")

image = testbed.render(768, 512, 1, True)

# write_image(out_dir + outname + ".png", image)

In [ ]:
??testbed

In [ ]:
!echo $CUDA_VISIBLE_DEVICES